<a href="https://colab.research.google.com/github/andreyxz2000/home/blob/main/%D0%90%D1%83%D0%B3%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%86%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image, ImageEnhance       # Инструменты для работы с изображениями
import matplotlib.pyplot as plt           # Отрисовка графиков
import numpy as np                        # Работа с массивами
import random                             # Генерация случайных чисел
import math                               # Математические функции
import cv2
import os
from google.colab import drive

# Подключение Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#сколько раз аугментируем каждое изображение
num_copy=10

# Путь к папке с изображениями на Google Drive
drive_image_folder = '/content/drive/MyDrive/Учебные БД/Исходные видео 2/Для аугментации'

# Путь к папке на Google Drive для сохранения преобразованных изображений (если не существует, то создаст ее)
colab_output_folder = '/content/drive/MyDrive/Учебные БД/Исходные видео 2/Аугментированное'

# Установить желаемые размеры изображений на выходе
width = 1920
height = 1080

In [ ]:
#функция аугментации
def augment_image(img,                    # Изображение для аугментации
                  ang=20,                  # Максимальный угол поворота
                  f_x=0.15,               # Максимальная подрезка по ширине
                  f_y=0.15,               # Максимальная подрезка по высоте
                  level_contr=0.3,        # Максимальное отклонение коэффициента контраста от нормы
                  level_brght=0.3):       # Максимальное отклонение коэффициента яркости от нормы


    # Функция нахождения ширины и высоты прямоугольника наибольшей площади
    # после поворота заданного прямоугольника на угол в градусах

    def rotated_rect(w, h, angle):
        angle = math.radians(angle)
        width_is_longer = w >= h
        side_long, side_short = (w,h) if width_is_longer else (h,w)

        sin_a, cos_a = abs(math.sin(angle)), abs(math.cos(angle))
        if side_short <= 2.*sin_a*cos_a*side_long or abs(sin_a-cos_a) < 1e-10:
            x = 0.5 * side_short
            wr, hr = (x/sin_a, x/cos_a) if width_is_longer else (x/cos_a, x/sin_a)
        else:
            cos_2a = cos_a*cos_a - sin_a*sin_a
            wr, hr = (w*cos_a - h*sin_a)/cos_2a, (h*cos_a - w*sin_a)/cos_2a

        return wr, hr


    # Функция случайной обрезки

    def random_crop(x,                    # Подаваемое изображение
                    f_x=f_x,              # Предел обрезки справа и слева (в масштабе ширины)
                    f_y=f_x               # Предел обрезки сверху и снизу (в масштабе высоты)
                    ):

        # Получение левой и правой границ обрезки
        left = x.width * random.random() * f_x
        right = x.width * (1. - random.random() * f_x) - 1.

        # Получение верхней и нижней границ обрезки
        upper = x.height * random.random() * f_y
        lower = x.height * (1. - random.random() * f_y) - 1.

        return x.crop((left, upper, right, lower))


    # Функция случайного поворота

    def random_rot(x,                     # Подаваемое изображение
                   ang=ang                # Максимальный угол поворота
                   ):

        # Случайное значение угла в диапазоне [-ang, ang]
        a = random.uniform(-1., 1.) * ang

        # Вращение картинки с расширением рамки
        r = x.rotate(a, expand=True)

        # Вычисление размеров прямоугольника обрезки максимальной площади
        # для размеров исходной картинки и угла поворота в градусах
        crop_w, crop_h = rotated_rect(x.width, x.height, a)

        # Обрезка повернутого изображения и возврат результата
        w, h = r.size
        return r.crop(((w - crop_w)*0.5, (h - crop_h)*0.5,
                       (w + crop_w)*0.5, (h + crop_h)*0.5))


    # Функция отражения

    def trans_img(x):
        return x.transpose(Image.FLIP_LEFT_RIGHT)


    # Функция случайного изменения контрастности

    def random_contrast(x,                   # Подаваемое изображение
                        level=level_contr    # Максимальное отклонение коэффициента контраста от нормы - число от 0. до 1.
                        ):

        enh = ImageEnhance.Contrast(x)       # Создание экземпляра класса Contrast
        factor = random.uniform(1. - level,
                                1. + level)  # Cлучайный коэффициент контраста из указанного интервала

        return enh.enhance(factor)           # Изменение коэффициента контраста



    # Функция случайного изменения яркости

    def random_brightness(x,                 # Подаваемое изображение
                        level=level_brght    # Максимальное отклонение коэффициента яркости от нормы - число от 0. до 1.
                        ):

        enh = ImageEnhance.Brightness(x)     # Создание экземпляра класса Brightness
        factor = random.uniform(1. - level,
                                1. + level)  # Cлучайный коэффициент контраста из указанного интервала

        return enh.enhance(factor)           # Изменение коэффициента яркости


    # Тело основной функции

    # Cоздание списка модификаций
    mod_oper = [random_rot,
                random_crop,
                trans_img,
                random_contrast,
                random_brightness]

    # Cлучайное количество изменений из списка; минимум одно изменение
    mod_count = random.randrange(len(mod_oper) + 1)

    # Случайный отбор индексов изменений в количестве mod_count без повторений
    mod_list = random.sample(range(len(mod_oper)), mod_count)

    # Применение модификаций по индексам из mod_list
    for mod_index in mod_list:
        img = mod_oper[mod_index](img)

    # Возврат результата
    return img


#функция отрисовки изображений
def show_image(img):
    plt.figure(figsize=(8, 5))            # Cоздание полотна для рисования
    plt.imshow(img)                       # Отрисовка изображения
    plt.axis('off')                       # Отключение ненужных осей
    plt.show()                            # Вывод результата

#функция ресайза аугментированных изображений (для приведения всех к одному формату (чтобы в yola не было глюков))
def resize_image(img, width, height):
    """
    Изменяет размер изображения на указанные ширину и высоту.

    :param img: Исходное изображение в формате numpy array.
    :param width: Новая ширина изображения.
    :param height: Новая высота изображения.
    :return: Измененное изображение.
    """
    resized_img = cv2.resize(img, (width, height))
    return resized_img




In [ ]:

# Создание папки на Colab, если она не существует
os.makedirs(colab_output_folder, exist_ok=True)

# Перечислить файлы в папке с изображениями
image_files = os.listdir(drive_image_folder)

# Цикл для обработки каждого изображения
for image_file in image_files:
    # Проверка, что это файл изображения
    if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        # Загрузить изображение с Google Drive
        image_path = os.path.join(drive_image_folder, image_file)
        img_start = Image.open(image_path)

        # Проверка наличия изображения и его размеров
        if img_start is not None and img_start.size[0] > 0 and img_start.size[1] > 0:
            # Провести необходимые преобразования (выполнить их 5 раз)
            for i in range(num_copy):
                # аугментация
                img_new = augment_image(img_start)


                # Проверка и преобразование img_new в формат numpy array, если это необходимо
                if not isinstance(img_new, np.ndarray):
                    try:
                        img_new = np.array(img_new)

                        # Проверяем, что img_new имеет 3 или 4 канала (цветное изображение)
                        if len(img_new.shape) == 3 and img_new.shape[2] in (3, 4):
                            # Преобразование в 32-битный формат с плавающей точкой, чтобы сохранить цвета
                            img_new = img_new.astype(np.float32)
                        else:
                            raise ValueError("img_new должно быть цветным изображением с 3 или 4 каналами.")
                    except Exception as e:
                        raise ValueError("Невозможно преобразовать img_new в формат numpy array.")

                resized_image = resize_image(img_new, width, height)
                # Указать путь и имя временного файла для сохранения в формате JPEG
                jpeg_image_path = f'/content/temp_{i}.jpg'

                # Сохранить изображение в формате JPEG с явным указанием цветового пространства (RGB)
                cv2.imwrite(jpeg_image_path, cv2.cvtColor(resized_image, cv2.COLOR_RGB2BGR))

                img  = Image.open(f'/content/temp_{i}.jpg')
                #show_image(img)
                #print('Размер аугментированного изображения:', img)

                # Создать имя файла для сохранения
                output_image_path = os.path.join(colab_output_folder, f'{image_file}_transformed_{i}.jpg')

                # Сохранить изображение в формате JPEG
                img.save(output_image_path)



        else:
            print(f"Изображение {image_file} имеет нулевые размеры или не существует. Пропускаем его.")
    else:
        print(f"Файл {image_file} не является изображением и будет пропущен.")
